In [12]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import numpy as np
import random
import os

# Set the random seed for reproducible results
SEED = 6
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Define transformations to include grayscale conversion
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
])

# Root directory for the dataset
root_dir = 'C:/Users/nici_/OneDrive/Desktop/CAS Machine Intelligence/Modul 2/Projekt/archive/Brain Tumor Data Set'

# Create the full dataset from both 'Healthy' and 'Brain Tumor' folders
full_dataset = datasets.ImageFolder(root=root_dir, transform=transform)

# Define the train and validation split (80/20)
train_size = int(0.8 * len(full_dataset))
validation_size = len(full_dataset) - train_size

# Split the dataset into training and validation datasets
train_dataset, val_dataset = random_split(full_dataset, [train_size, validation_size], generator=torch.Generator().manual_seed(SEED))

# Create the DataLoaders for training and validation sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)

In [13]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.datasets import ImageFolder
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR

class ModifiedCNN(nn.Module):
    def __init__(self):
        super(ModifiedCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(in_features=64*32*32, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=2) 
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2)
        x = self.dropout(x)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2)
        x = self.dropout(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Initialize the model
model = ModifiedCNN()

# Define loss criterion
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = Adam(model.parameters(), lr=0.001)

# After defining your optimizer, create the scheduler
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)
num_epochs = 8

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    # Validation step
    model.eval()
    val_running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    with torch.no_grad():  
        for inputs, targets in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_predictions += targets.size(0)
            correct_predictions += (predicted == targets).sum().item()
            val_loss = criterion(outputs, targets)
            val_running_loss += val_loss.item()

    # Calculate losses and accuracy
    epoch_loss = running_loss / len(train_loader)
    val_epoch_loss = val_running_loss / len(val_loader)
    val_accuracy = (correct_predictions / total_predictions) * 100

    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Train Loss: {epoch_loss:.4f}, '
          f'Validation Loss: {val_epoch_loss:.4f}, '
          f'Validation Accuracy: {val_accuracy:.2f}%')
    
    # Step the scheduler
    scheduler.step()  

print('Training and validation complete.')

Epoch [1/8], Train Loss: 0.7401, Validation Loss: 0.6728, Validation Accuracy: 67.17%
Epoch [2/8], Train Loss: 0.6017, Validation Loss: 0.5555, Validation Accuracy: 76.09%
Epoch [3/8], Train Loss: 0.5053, Validation Loss: 0.4563, Validation Accuracy: 77.61%
Epoch [4/8], Train Loss: 0.4805, Validation Loss: 0.4475, Validation Accuracy: 77.61%
Epoch [5/8], Train Loss: 0.4414, Validation Loss: 0.3695, Validation Accuracy: 82.17%
Epoch [6/8], Train Loss: 0.3943, Validation Loss: 0.3585, Validation Accuracy: 83.70%
Epoch [7/8], Train Loss: 0.3750, Validation Loss: 0.3423, Validation Accuracy: 84.57%
Epoch [8/8], Train Loss: 0.3639, Validation Loss: 0.3318, Validation Accuracy: 85.33%
Training and validation complete.
